Making sure all errors are suppressed.

In [1]:
import warnings
warnings.filterwarnings("ignore") 

In [2]:
#Importing libraries
import pandas as pd
import os
import numpy as np
import copy
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew
# from sklearn.linear_model import LinearRegression
%matplotlib inline

In [7]:
#Data Directory where you have data for all the indicators.
filLoc = "..//Dataset"

In [8]:
#function to calculate the slope - trend magnitude for availability, accessibility, and utilisation stability

def retSlope(de):
    X = np.array(de["Year"]).reshape(-1, 1)
    y = de["Value"]
    model = LinearRegression().fit(X, y)
    return model.coef_[0]
    

In [9]:
#function to calculate the standard deviation
def retStd(de):
    return np.round(de["Value"].std())
    

Iterating through each indicator and appending them to form a master dataset
For the prevalence of undernourishment indicator, all the values are scaled based on the regional average in 2015.

In [10]:
df =pd.DataFrame(columns = ["Country","Year","Value","Indicator"])

with os.scandir(filLoc) as root_dir:
    for path in root_dir:
         if path.is_file():
            print(path.name.split('.')[0])
           
            data = pd.read_csv(path,encoding = 'latin-1')
            data['Country'] = data["Country"].str.lstrip()
            cols = [i for i in data.columns if i!="Country"]
            
#             if path.name.split('.')[0] in ['Prevelance of Undernourishment','Proportion of children moderately or severely wasted','Proportion of children moderately or severely stunted']:
#                 print(cols)
            if path.name.split('.')[0] in ['Prevalence of undernourishment']:

                average = data[str(2015)].mean()
                data[str(2015)]=np.round(data[str(2015)]/average*100,2)
                for k in cols:
                    if k!="2015":
                        data[k] = data[k]/data[str(2015)]*100

#                 print(data.head())
            
            temp = pd.melt(data,id_vars=['Country'], var_name = "Year", value_vars=cols,value_name = "Value")
            temp["Indicator"] = path.name.split('.')[0]
            
            df = df.append(temp)
#             print(df.tail(5))
            

Access to drinking water and sanitation
Access to electricity
Accessibility trend
Availability trend
Average dietary energy adequacy
Average protein supply
Direct disaster economic loss
Food import dependency ratio
Governance indicator
Inflation rate
Per capita GAP
Population covered by at least one social protection system
Poverty population proportion
Prevalence of adult obesity
Prevalence of undernourishment
Proportion of children moderately or severely stunted
Proportion of children moderately or severely wasted
Proportion of severely food insecure
Remittances
Rural access index
Score for adoption and implementation of disaster reduction strategies
Unemployment rate
Utilization trend


In [11]:
#This just adds Pacific as a country to keep track of the region.

temper = df.groupby(["Indicator","Year"])["Value"].mean().reset_index()
temper["Country"]="Pacific"

In [15]:
df1 = df.append(temper)
df1.tail(5)

,Country,Year,Value,Indicator
413,Pacific,2018,11.235000,Unemployment rate
414,Pacific,2019,21.902000,Unemployment rate
415,Pacific,2020,29.770000,Unemployment rate
416,Pacific,Trend,0.037066,Utilization trend
417,Pacific,Variability,1.125000,Utilization trend


In [16]:
#Saving the file just as a backup
df1["Value"] = np.round(df1["Value"],2)
df1.dropna(subset = ["Country"]).to_csv("..//Data.csv", index = False)


In [17]:
#Reading from the back up file for further data
test = pd.read_csv("..//Data.csv")
test.tail()

,Country,Year,Value,Indicator
6479,Pacific,2018,11.24,Unemployment rate
6480,Pacific,2019,21.90,Unemployment rate
6481,Pacific,2020,29.77,Unemployment rate
6482,Pacific,Trend,0.04,Utilization trend
6483,Pacific,Variability,1.12,Utilization trend


This is the entry point if you want to change the baseline for any indicator. If you are adding any new indicator to the dashboard, you need to add the baseline here as well.
There are two natures of indicator - one where higher values than a baseline are desirable (eg Average dietary energy adequacy, remittances) and the other (Proportion of food insecurity, Inflation rate) where lower values than a baseline are desirable.
For the first kind (higher values desirable), the baselines are defined as the last element of the list where the first element has to be less than baseline while for the second kind (lower values desirable), the baselines are defined as the first element and the last element has to be less than the baseline.

For a range of values for mitigator (good), special declaration needs to be done. Check the case for average protein supply.

In [20]:
col_dic = {
    'Access to drinking water and sanitation':[75,100],
    'Access to electricity':[75,100],
    'Availability trend_t': [0,-1],
    'Availability trend_v': [50,34],
    'Accessibility trend_t': [0,-1],
    'Accessibility trend_v': [50,2],
    'Average dietary energy adequacy':[75,100],
    'Average protein supply':[60,80],
    'Direct disaster economic loss':[0.05,0],
#     'Share of energy derived from cereals, tubers, and roots':[50,30],
    'Food import dependency ratio':[25,5],
    'Governance indicator':[-1,0],
    'Per capita GAP':[50,100],
    'Inflation rate':[3,1],
    'Poverty population proportion':[10,5],
    'Prevalence of undernourishment':[100,80],
#     'Proportion of children moderately or severely stunted':[5,4],
    'Proportion of children moderately or severely stunted':[3,2],
    'Proportion of children moderately or severely wasted':[5,3],
    'Proportion of severely food insecure':[5,3],
    'Remittances':[5,10],
    'Rural access index': [0.6,0.75],
    'Score for adoption and implementation of disaster reduction strategies':[-1,0],
    'Unemployment rate':[6,3],
    'Utilization trend_t': [0,-1],
    'Utilization trend_v': [5,1],
    'Prevalence of adult obesity':[5,4],
    'Population covered by at least one social protection system':[75,100]
    
    
}

In [21]:
#Function to return color based on the country value and the baseline.
def retColor(x,col_dic):
    if pd.isna(x["Value"]):
        return "gray"
    else:
        val=None
        if x["Indicator"] in ["Availability trend", "Accessibility trend","Utilization trend"]:
            if x["Year"]=="Trend":
                val = col_dic[x["Indicator"]+"_t"]
            else:
                 val = col_dic[x["Indicator"]+"_v"]
        else:
            val = col_dic[x["Indicator"]]
        if x["Indicator"]=='Average protein supply':
            if (x["Value"]>=val[0] and x["Value"]<=val[-1]):
                return "green"
            else:
                return "red"
        else:
            if val[0]<val[-1]:
                if x["Value"]>=val[-1]:
                    return "green"
                else:
                    return "red"
            else:
                if x["Value"]<val[-1]:
                    return "green"
                else:
                    return "red"

In [22]:
#function to return the value of baseline for the indicator
def retBaseline(x,col_dic):
#         val = col_dic[x["Indicator"]]
    val=None
    if x["Indicator"] in ["Availability trend", "Accessibility trend","Utilization trend"]:
        if x["Year"]=="Trend":
            val = col_dic[x["Indicator"]+"_t"]
        else:
             val = col_dic[x["Indicator"]+"_v"]
    else:
        val = col_dic[x["Indicator"]]
        
    if x["Indicator"]=='Average protein supply':
        return '{},{}'.format(val[0],val[-1])

    else:

        if val[0]<val[-1]:
            return val[-1]
        else:
            return val[0]


In [23]:
#function to return the zone for amplifier/mitigator
def retUpDown(x,col_dic):
    val=None
    if x["Indicator"] in ["Availability trend", "Accessibility trend","Utilization trend"]:
        if x["Year"]=="Trend":
            val = col_dic[x["Indicator"]+"_t"]
        else:
             val = col_dic[x["Indicator"]+"_v"]
    else:
        val = col_dic[x["Indicator"]]
    if x["Indicator"]=='Average protein supply':
        return "Range"
    else:
        if val[0]<val[-1]:
            return "Up"
        else:
            return "Down"

In [24]:
#Applying these three functions for additional info
test["Color"]= test.apply(lambda x: retColor(x,col_dic), axis =1)
test["Baseline"]= test.apply(lambda x: retBaseline(x,col_dic), axis =1)
test["UpDown"]= test.apply(lambda x: retUpDown(x,col_dic), axis =1)

In [25]:
#Checking the values of baseline and other additional info
for i in test["Indicator"].unique():
    print(test[test["Indicator"]==i].head(1))

          Country  Year  Value                                Indicator  \
0  American Samoa  2000   81.5  Access to drinking water and sanitation   

   Color Baseline UpDown  
0  green       80     Up  
    Country  Year  Value              Indicator Color Baseline UpDown
441    Fiji  1961  71.77  Access to electricity   red      100     Up
             Country   Year  Value            Indicator Color Baseline UpDown
1401  American Samoa  Trend  -0.39  Accessibility trend   red        0   Down
           Country   Year  Value           Indicator  Color Baseline UpDown
1443  Cook Islands  Trend  -6.16  Availability trend  green        0   Down
             Country  Year  Value                        Indicator Color  \
1475  American Samoa  2001    NaN  Average dietary energy adequacy  gray   

     Baseline UpDown  
1475      100     Up  
             Country  Year  Value               Indicator Color Baseline  \
1835  American Samoa  2001    NaN  Average protein supply  gray    60,80

In [26]:
#writing the data to file
test.to_csv("..//Data1.csv",index = False)